Reads in raw all_league_match_data, extracts 'incomplete' and fixtures next 7 days - stores as 'temp'.   
Remaining script engineers HG_Exp and AG_Exp, saves as next_fixts (processed)


In [65]:

import pandas as pd
import time
import os
import unidecode
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from warnings import filterwarnings
filterwarnings('ignore')

In [66]:
df=pd.read_csv(r'C:\Users\MikeD\Streamlit_BK_App\data\raw\all_leagues_match_data_raw.csv')

In [67]:
df.head()

,id,homeID,awayID,season,status,roundID,game_week,revised_game_week,homeGoals,awayGoals,homeGoalCount,awayGoalCount,totalGoalCount,team_a_corners,team_b_corners,totalCornerCount,team_a_offsides,team_b_offsides,team_a_yellow_cards,team_b_yellow_cards,team_a_red_cards,team_b_red_cards,team_a_shotsOnTarget,team_b_shotsOnTarget,team_a_shotsOffTarget,team_b_shotsOffTarget,team_a_shots,team_b_shots,team_a_fouls,team_b_fouls,team_a_possession,team_b_possession,refereeID,coach_a_ID,coach_b_ID,stadium_name,stadium_location,team_a_cards_num,team_b_cards_num,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over05,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_ft_under05,odds_ft_under15,odds_ft_under25,odds_ft_under35,odds_ft_under45,odds_btts_yes,odds_btts_no,odds_team_a_cs_yes,odds_team_a_cs_no,odds_team_b_cs_yes,odds_team_b_cs_no,odds_doublechance_1x,odds_doublechance_12,odds_doublechance_x2,odds_1st_half_result_1,odds_1st_half_result_x,odds_1st_half_result_2,odds_2nd_half_result_1,odds_2nd_half_result_x,odds_2nd_half_result_2,odds_dnb_1,odds_dnb_2,odds_corners_over_75,odds_corners_over_85,odds_corners_over_95,odds_corners_over_105,odds_corners_over_115,odds_corners_under_75,odds_corners_under_85,odds_corners_under_95,odds_corners_under_105,odds_corners_under_115,odds_corners_1,odds_corners_x,odds_corners_2,odds_team_to_score_first_1,odds_team_to_score_first_x,odds_team_to_score_first_2,odds_win_to_nil_1,odds_win_to_nil_2,odds_1st_half_over05,odds_1st_half_over15,odds_1st_half_over25,odds_1st_half_over35,odds_1st_half_under05,odds_1st_half_under15,odds_1st_half_under25,odds_1st_half_under35,odds_2nd_half_over05,odds_2nd_half_over15,odds_2nd_half_over25,odds_2nd_half_over35,odds_2nd_half_under05,odds_2nd_half_under15,odds_2nd_half_under25,odds_2nd_half_under35,odds_btts_1st_half_yes,odds_btts_1st_half_no,odds_btts_2nd_half_yes,odds_btts_2nd_half_no,overallGoalCount,ht_goals_team_a,ht_goals_team_b,goals_2hg_team_a,goals_2hg_team_b,GoalCount_2hg,HTGoalCount,date_unix,winningTeam,no_home_away,btts_potential,btts_fhg_potential,btts_2hg_potential,goalTimingDisabled,attendance,corner_timings_recorded,card_timings_recorded,team_a_fh_corners,team_b_fh_corners,team_a_2h_corners,team_b_2h_corners,corner_fh_count,corner_2h_count,team_a_fh_cards,team_b_fh_cards,team_a_2h_cards,team_b_2h_cards,total_fh_cards,total_2h_cards,attacks_recorded,team_a_dangerous_attacks,team_b_dangerous_attacks,team_a_attacks,team_b_attacks,team_a_xg,team_b_xg,total_xg,team_a_penalties_won,team_b_penalties_won,team_a_penalty_goals,team_b_penalty_goals,team_a_penalty_missed,team_b_penalty_missed,pens_recorded,goal_timings_recorded,team_a_0_10_min_goals,team_b_0_10_min_goals,team_a_corners_0_10_min,team_b_corners_0_10_min,team_a_cards_0_10_min,team_b_cards_0_10_min,throwins_recorded,team_a_throwins,team_b_throwins,freekicks_recorded,team_a_freekicks,team_b_freekicks,goalkicks_recorded,team_a_goalkicks,team_b_goalkicks,o45_potential,o35_potential,o25_potential,o15_potential,o05_potential,o15HT_potential,o05HT_potential,o05_2H_potential,o15_2H_potential,corners_potential,offsides_potential,cards_potential,avg_potential,home_url,home_image,home_name,away_url,away_image,away_name,home_ppg,away_ppg,pre_match_home_ppg,pre_match_away_ppg,pre_match_teamA_overall_ppg,pre_match_teamB_overall_ppg,u45_potential,u35_potential,u25_potential,u15_potential,u05_potential,corners_o85_potential,corners_o95_potential,corners_o105_potential,team_a_xg_prematch,team_b_xg_prematch,total_xg_prematch,match_url,competition_id,matches_completed_minimum,over05,over15,over25,over35,over45,over55,btts,homeGoals_timings,awayGoals_timings,league_name
0,7466677,149,162,2024/2025,complete,110026,1,-1,['87'],[],1,0,1,7,8,15,3,1,2,3,0,0,6,3,7,4,15,12,12,10,55,45,1248.0,879.0,214.0,Old Trafford (Manchester),NaN,2,3,1.48,4.42,6.65,1.01,1.14,1.58,2.28,3.70,23.0,6.20,2.39,1.56,1.30,1.57,2.25,2.75,1.40,7.0,1.07,1.16,1.20,2.55,2.05,2.60,5.0,1.80,3.05,4.75,0,0,1.14,1.26,1.44,1.70,2.06,4.80,3.40,2.55,2.00,1.66,

In [68]:
for col in df.columns:
    print(col)

id
homeID
awayID
season
status
roundID
game_week
revised_game_week
homeGoals
awayGoals
homeGoalCount
awayGoalCount
totalGoalCount
team_a_corners
team_b_corners
totalCornerCount
team_a_offsides
team_b_offsides
team_a_yellow_cards
team_b_yellow_cards
team_a_red_cards
team_b_red_cards
team_a_shotsOnTarget
team_b_shotsOnTarget
team_a_shotsOffTarget
team_b_shotsOffTarget
team_a_shots
team_b_shots
team_a_fouls
team_b_fouls
team_a_possession
team_b_possession
refereeID
coach_a_ID
coach_b_ID
stadium_name
stadium_location
team_a_cards_num
team_b_cards_num
odds_ft_1
odds_ft_x
odds_ft_2
odds_ft_over05
odds_ft_over15
odds_ft_over25
odds_ft_over35
odds_ft_over45
odds_ft_under05
odds_ft_under15
odds_ft_under25
odds_ft_under35
odds_ft_under45
odds_btts_yes
odds_btts_no
odds_team_a_cs_yes
odds_team_a_cs_no
odds_team_b_cs_yes
odds_team_b_cs_no
odds_doublechance_1x
odds_doublechance_12
odds_doublechance_x2
odds_1st_half_result_1
odds_1st_half_result_x
odds_1st_half_result_2
odds_2nd_half_result_1
odds_2

In [69]:
# remove foreign lettering - Home, store as new column
new_team_names_h = []
for name in df['home_name']:
    name = unidecode.unidecode(name)
    new_team_names_h.append(name)
df['Home_Name'] = new_team_names_h

# remove foreign lettering - Away, store as new column
new_team_names_a = []
for name in df['away_name']:
    name = unidecode.unidecode(name)
    new_team_names_a.append(name)
df['Away_Name'] = new_team_names_a

# drop original columns which have foreign accented team names
df.drop(['home_name', 'away_name'], axis=1, inplace=True)

# rename latin team name column back to original headers
df.rename(columns={
    'Home_Name': 'home_name',
    'Away_Name': 'away_name', 
      }, inplace=True)

In [70]:
# extract matches not played yet - status = incomplete
df_incomplete = df[df['status'] == 'incomplete']

In [71]:
df_incomplete.head()

,id,homeID,awayID,season,status,roundID,game_week,revised_game_week,homeGoals,awayGoals,homeGoalCount,awayGoalCount,totalGoalCount,team_a_corners,team_b_corners,totalCornerCount,team_a_offsides,team_b_offsides,team_a_yellow_cards,team_b_yellow_cards,team_a_red_cards,team_b_red_cards,team_a_shotsOnTarget,team_b_shotsOnTarget,team_a_shotsOffTarget,team_b_shotsOffTarget,team_a_shots,team_b_shots,team_a_fouls,team_b_fouls,team_a_possession,team_b_possession,refereeID,coach_a_ID,coach_b_ID,stadium_name,stadium_location,team_a_cards_num,team_b_cards_num,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over05,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_ft_under05,odds_ft_under15,odds_ft_under25,odds_ft_under35,odds_ft_under45,odds_btts_yes,odds_btts_no,odds_team_a_cs_yes,odds_team_a_cs_no,odds_team_b_cs_yes,odds_team_b_cs_no,odds_doublechance_1x,odds_doublechance_12,odds_doublechance_x2,odds_1st_half_result_1,odds_1st_half_result_x,odds_1st_half_result_2,odds_2nd_half_result_1,odds_2nd_half_result_x,odds_2nd_half_result_2,odds_dnb_1,odds_dnb_2,odds_corners_over_75,odds_corners_over_85,odds_corners_over_95,odds_corners_over_105,odds_corners_over_115,odds_corners_under_75,odds_corners_under_85,odds_corners_under_95,odds_corners_under_105,odds_corners_under_115,odds_corners_1,odds_corners_x,odds_corners_2,odds_team_to_score_first_1,odds_team_to_score_first_x,odds_team_to_score_first_2,odds_win_to_nil_1,odds_win_to_nil_2,odds_1st_half_over05,odds_1st_half_over15,odds_1st_half_over25,odds_1st_half_over35,odds_1st_half_under05,odds_1st_half_under15,odds_1st_half_under25,odds_1st_half_under35,odds_2nd_half_over05,odds_2nd_half_over15,odds_2nd_half_over25,odds_2nd_half_over35,odds_2nd_half_under05,odds_2nd_half_under15,odds_2nd_half_under25,odds_2nd_half_under35,odds_btts_1st_half_yes,odds_btts_1st_half_no,odds_btts_2nd_half_yes,odds_btts_2nd_half_no,overallGoalCount,ht_goals_team_a,ht_goals_team_b,goals_2hg_team_a,goals_2hg_team_b,GoalCount_2hg,HTGoalCount,date_unix,winningTeam,no_home_away,btts_potential,btts_fhg_potential,btts_2hg_potential,goalTimingDisabled,attendance,corner_timings_recorded,card_timings_recorded,team_a_fh_corners,team_b_fh_corners,team_a_2h_corners,team_b_2h_corners,corner_fh_count,corner_2h_count,team_a_fh_cards,team_b_fh_cards,team_a_2h_cards,team_b_2h_cards,total_fh_cards,total_2h_cards,attacks_recorded,team_a_dangerous_attacks,team_b_dangerous_attacks,team_a_attacks,team_b_attacks,team_a_xg,team_b_xg,total_xg,team_a_penalties_won,team_b_penalties_won,team_a_penalty_goals,team_b_penalty_goals,team_a_penalty_missed,team_b_penalty_missed,pens_recorded,goal_timings_recorded,team_a_0_10_min_goals,team_b_0_10_min_goals,team_a_corners_0_10_min,team_b_corners_0_10_min,team_a_cards_0_10_min,team_b_cards_0_10_min,throwins_recorded,team_a_throwins,team_b_throwins,freekicks_recorded,team_a_freekicks,team_b_freekicks,goalkicks_recorded,team_a_goalkicks,team_b_goalkicks,o45_potential,o35_potential,o25_potential,o15_potential,o05_potential,o15HT_potential,o05HT_potential,o05_2H_potential,o15_2H_potential,corners_potential,offsides_potential,cards_potential,avg_potential,home_url,home_image,away_url,away_image,home_ppg,away_ppg,pre_match_home_ppg,pre_match_away_ppg,pre_match_teamA_overall_ppg,pre_match_teamB_overall_ppg,u45_potential,u35_potential,u25_potential,u15_potential,u05_potential,corners_o85_potential,corners_o95_potential,corners_o105_potential,team_a_xg_prematch,team_b_xg_prematch,total_xg_prematch,match_url,competition_id,matches_completed_minimum,over05,over15,over25,over35,over45,over55,btts,homeGoals_timings,awayGoals_timings,league_name,home_name,away_name
70,7466747,148,59,2024/2025,incomplete,110026,8,-1,[],[],0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1248.0,-1.0,-1.0,"Vitality Stadium (Bournemouth, Dorset)",NaN,-1,-1,6.01,1.89,2.42,1.04,1.23,1.62,2.36,5.40,15.0,4.35,2.25,1.59,1.17,1.70,2.05,5.25,1.12,2.55,1.45,2.23,1.24,1.20,4.75,2.30,2.25,4.33,2.75,1.95,0,0,1.23,1.41,1.66,2.04,2.5

In [72]:
# extract required columns
df_incomplete_2 = df_incomplete[['league_name',
                                  'season',
                                  'date_unix',
                                    'game_week',
                                      'home_name',
                                        'away_name',
                                          'odds_ft_1',
                                            'odds_ft_x',
                                              'odds_ft_2',
                                                'odds_ft_over25',
                                                'odds_ft_under25',
                                                 'odds_corners_over_105',
                                                  'odds_corners_under_105'
                                                    ]]

In [73]:
df_incomplete_2.head()

,league_name,season,date_unix,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,odds_corners_over_105,odds_corners_under_105
70,eng1,2024/2025,1729355400,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2.04,1.68
71,eng1,2024/2025,1729346400,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2.08,1.65
72,eng1,2024/2025,1729346400,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,1.82,1.86
73,eng1,2024/2025,1729438200,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,0.00,0.00
74,eng1,2024/2025,1729346400,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,1.73,1.96


In [74]:
# transform 'date_unix' to datetime
df_incomplete_2['datetime'] = pd.to_datetime(df_incomplete_2['date_unix'], unit='s')
df_incomplete_2['date'] = df_incomplete_2['datetime'].dt.date

In [75]:
df_incomplete_2.drop(['datetime', 'date_unix'],axis=1, inplace=True)


In [76]:

df_incomplete_2.head()

,league_name,season,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,odds_corners_over_105,odds_corners_under_105,date
70,eng1,2024/2025,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2.04,1.68,2024-10-19
71,eng1,2024/2025,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2.08,1.65,2024-10-19
72,eng1,2024/2025,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,1.82,1.86,2024-10-19
73,eng1,2024/2025,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,0.00,0.00,2024-10-20
74,eng1,2024/2025,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,1.73,1.96,2024-10-19


In [77]:
# # Convert the 'date' column to datetime and normalize (set time to 00:00)
# df_incomplete_2['date'] = pd.to_datetime(df_incomplete_2['date'])

# Get the current date and calculate the end of the next 7 days
today = pd.Timestamp('today').normalize()  # Normalize to remove the time part
next_week = today + pd.Timedelta(days=7)  # 7 full days from today

# # Check date values for debugging
# print("Date range: Today =", today, "to", next_week_end)

# # Filter rows between today and the end of the next 7 days (inclusive)
# df_fixts = df_incomplete_2[(df_incomplete_2['date'] >= today) & (df_incomplete_2['date'] < next_week_end)]

# df_test = df_incomplete_2[df_incomplete_2['date'] <= next_week]
df_fixts = df_incomplete_2[(pd.to_datetime(df_incomplete_2['date']) >= today) & 
                            (pd.to_datetime(df_incomplete_2['date']) < next_week)]



In [78]:

df_fixts.head()

,league_name,season,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,odds_corners_over_105,odds_corners_under_105,date
70,eng1,2024/2025,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2.04,1.68,2024-10-19
71,eng1,2024/2025,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2.08,1.65,2024-10-19
72,eng1,2024/2025,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,1.82,1.86,2024-10-19
73,eng1,2024/2025,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,0.00,0.00,2024-10-20
74,eng1,2024/2025,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,1.73,1.96,2024-10-19


____________________________________________________________________________________-

In [79]:
# save this file as **temporary** to read in again for future code tweaking - above code is set to use today() to extract fixtures 
df_fixts.to_csv(r'C:\Users\MikeD\Streamlit_BK_App\data\processed\next_fixts_TEMP.csv', index=False)
print('Saved sucessfully')

Saved sucessfully


In [80]:
df_fixts = pd.read_csv(r'C:\Users\MikeD\Streamlit_BK_App\data\processed\next_fixts_TEMP.csv')

In [81]:
df_fixts.head()

,league_name,season,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,odds_corners_over_105,odds_corners_under_105,date
0,eng1,2024/2025,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2.04,1.68,2024-10-19
1,eng1,2024/2025,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2.08,1.65,2024-10-19
2,eng1,2024/2025,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,1.82,1.86,2024-10-19
3,eng1,2024/2025,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,0.00,0.00,2024-10-20
4,eng1,2024/2025,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,1.73,1.96,2024-10-19


In [82]:
df_fixts['HW_%'] = round((1/df_fixts['odds_ft_1']) / (1/df_fixts['odds_ft_1'] + 1/df_fixts['odds_ft_x'] + 1/df_fixts['odds_ft_2']), 2)
df_fixts['X_%'] = round((1/df_fixts['odds_ft_x']) / (1/df_fixts['odds_ft_1'] + 1/df_fixts['odds_ft_x'] + 1/df_fixts['odds_ft_2']), 2)
df_fixts['AW_%'] = round((1/df_fixts['odds_ft_2']) / (1/df_fixts['odds_ft_1'] + 1/df_fixts['odds_ft_x'] + 1/df_fixts['odds_ft_2']), 2)

df_fixts['Ov2.5_%'] = round((1/df_fixts['odds_ft_over25']) / (1/df_fixts['odds_ft_over25'] + 1/df_fixts['odds_ft_under25']), 2)
df_fixts['Un2.5_%'] = round((1/df_fixts['odds_ft_under25']) / (1/df_fixts['odds_ft_over25'] + 1/df_fixts['odds_ft_under25']), 2)

# df_fixts['Ov10.5_C_%'] = round((1/df_fixts['odds_corners_over_105']) / (1/df_fixts['odds_corners_over_105'] + 1/df_fixts['odds_corners_under_105']), 2)
# df_fixts['Un10.5_C_%'] = round((1/df_fixts['odds_corners_under_105']) / (1/df_fixts['odds_corners_over_105'] + 1/df_fixts['odds_corners_under_105']), 2)

In [83]:
df_fixts.head()

,league_name,season,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,odds_corners_over_105,odds_corners_under_105,date,HW_%,X_%,AW_%,Ov2.5_%,Un2.5_%
0,eng1,2024/2025,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2.04,1.68,2024-10-19,0.15,0.48,0.37,0.58,0.42
1,eng1,2024/2025,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2.08,1.65,2024-10-19,0.40,0.27,0.33,0.53,0.47
2,eng1,2024/2025,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,1.82,1.86,2024-10-19,0.40,0.28,0.32,0.52,0.48
3,eng1,2024/2025,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,0.00,0.00,2024-10-20,0.67,0.22,0.11,0.70,0.30
4,eng1,2024/2025,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,1.73,1.96,2024-10-19,0.29,0.50,0.21,0.60,0.40


In [84]:
df_ou = pd.read_csv(r'C:\Users\MikeD\Streamlit_BK_App\OverUnder_Exp.csv')
# df_ou_10_5 = pd.read_csv(r'C:\Users\MikeD\Streamlit_BK_App\OverUnder_10.5_Exp.csv')
df_dnb = pd.read_csv(r'C:\Users\MikeD\Streamlit_BK_App\DNB_Sup_Conversion.csv')

df_dnb.drop(['dnb price'], axis=1, inplace=True)

df_ou.drop(['Exp1', 'Under', 'Un2.5_%'], axis=1, inplace=True)
df_ou.drop_duplicates(subset='Ov2.5_%', inplace=True)

# df_ou_10_5.drop(['Exp1', 'Under', 'Un10.5_%'], axis=1, inplace=True)
# df_ou_10_5.drop_duplicates(subset='Ov10.5_%', inplace=True)


In [85]:
df_ou.head()

,Over,Ov2.5_%,Exp
0,4.45,0.22,1.64
1,4.33,0.23,1.66
2,4.23,0.24,1.68
4,4.03,0.25,1.73
6,3.86,0.26,1.77


In [86]:
df_fixts.head()

,league_name,season,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,odds_corners_over_105,odds_corners_under_105,date,HW_%,X_%,AW_%,Ov2.5_%,Un2.5_%
0,eng1,2024/2025,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2.04,1.68,2024-10-19,0.15,0.48,0.37,0.58,0.42
1,eng1,2024/2025,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2.08,1.65,2024-10-19,0.40,0.27,0.33,0.53,0.47
2,eng1,2024/2025,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,1.82,1.86,2024-10-19,0.40,0.28,0.32,0.52,0.48
3,eng1,2024/2025,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,0.00,0.00,2024-10-20,0.67,0.22,0.11,0.70,0.30
4,eng1,2024/2025,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,1.73,1.96,2024-10-19,0.29,0.50,0.21,0.60,0.40


In [87]:
df_fixts.drop(['odds_corners_over_105', 'odds_corners_under_105'], axis=1, inplace=True)

In [88]:
# when corners available, merge 'df_ov_un_10.5' also
df2 = pd.merge(left=df_fixts, right=df_ou,how='left')
df2.drop_duplicates(keep='first', inplace=True)

In [89]:
df2.head()

,league_name,season,game_week,home_name,away_name,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over25,odds_ft_under25,date,HW_%,X_%,AW_%,Ov2.5_%,Un2.5_%,Over,Exp
0,eng1,2024/2025,8,AFC Bournemouth,Arsenal,6.01,1.89,2.42,1.62,2.25,2024-10-19,0.15,0.48,0.37,0.58,0.42,1.74,3.03
1,eng1,2024/2025,8,Fulham,Aston Villa,2.44,3.52,2.92,1.80,2.00,2024-10-19,0.40,0.27,0.33,0.53,0.47,1.90,2.79
2,eng1,2024/2025,8,Ipswich Town,Everton,2.42,3.43,3.01,1.83,1.95,2024-10-19,0.40,0.28,0.32,0.52,0.48,1.94,2.76
3,eng1,2024/2025,8,Liverpool,Chelsea,1.35,4.04,7.99,1.36,3.10,2024-10-20,0.67,0.22,0.11,0.70,0.30,1.43,3.63
4,eng1,2024/2025,8,Manchester United,Brentford,3.07,1.80,4.39,1.57,2.35,2024-10-19,0.29,0.50,0.21,0.60,0.40,1.68,3.10


In [90]:
df2['1x2_%'] = df2['HW_%'] + df2['X_%'] + df2['AW_%']
df2['H_true'] = df2['odds_ft_1'] * df2['1x2_%']
df2['X_true'] = df2['odds_ft_x'] * df2['1x2_%']
df2['A_true'] = df2['odds_ft_2'] * df2['1x2_%']
df2['dnb %'] = round(1/df2['H_true'] / (1/df2['H_true'] + 1/df2['A_true']), 2)

In [91]:
df_dnb.head()

,dnb %,Sup
0,0.02,-2.6
1,0.03,-2.5
2,0.04,-2.4
3,0.05,-2.3
4,0.06,-2.2


In [92]:
df3 = pd.merge(left=df2, right=df_dnb, how='left')

In [93]:
df3['HG_Exp'] = (df3['Exp'] / 2) + 0.5 * df3['Sup']
df3['AG_Exp'] = (df3['Exp'] / 2) - 0.5 * df3['Sup']

In [94]:
df4 = df3[['league_name', 'season', 'date', 'home_name', 'away_name', 'odds_ft_1', 'odds_ft_x', 'odds_ft_2','HG_Exp', 'AG_Exp']]

In [95]:
# save to processed
df4.to_csv(r'C:\Users\MikeD\Streamlit_BK_App\data\processed\next_fixts.csv', index=False)
print('Saved sucessfully')

Saved sucessfully
